In [1]:
from pymongo import MongoClient
import pprint

import pandas as pd

import numpy as np

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

import json
import time

from urllib.request import Request, urlopen

In [3]:
client = MongoClient('localhost', 27017)
db_gold = client['gold_scraper']

prices = db_gold['prices']

In [4]:
url = 'https://www.usagold.com/reference/prices/goldhistory.php?ddYears=2019'

In [5]:
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()

In [6]:
soup = BeautifulSoup(webpage, 'html.parser')

In [7]:
links = soup.find_all('tr')

In [8]:
len(links)

278

In [11]:
def scrape_gold(years):
    """
    PARAMETERS:
    years - tuple, start and end year of interest (end year exclusive) 
    
    RETURNS:
    None - uploads rows of rate data to MongoDB
    """


    #web pages are divided up by year. Iterate through years to generate proper soup
    for year in range(years[0], years[1]):

        url = f'https://www.usagold.com/reference/prices/goldhistory.php?ddYears={year}'
        req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'html.parser')

        # parse out all table rows
        links = soup.find_all('tr')

        for link in links:
            d = dict()
            for idx, i in enumerate(link.find_all('td')):

                if idx == 0:
                    d['date'] = i.text

                elif idx == 1:
                    d['gold_price_usd'] = i.text

            prices.insert_one(d)

        time.sleep(5)

    return None

In [12]:
scrape_gold((1990,2021))